# COMPO CREATION
#### The COMPO Index is an input required for 
#### -(F) summer change classification
#### -(G) confidence and priority computation (year report)

In [ ]:
import os, sys, datetime
sys.path.append("/home/gswinnen/SARSAR_Package_RenPri/code/") # emplacement des modules RenPri

#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/")                   # emplacement des modules RenPri
#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/rme_chg_detection_module/") # emplacement de la fonction de Mattia

from issep import sarsar_admin
from os.path import join
from lecture_ini import config
from csv import reader

In [ ]:
# Reading of the list of SAR without S2 data
with open("/home/gswinnen/Public/liste_sites_sans_NDVI.txt",'r') as file:
    csv_reader = reader(file)
    list_lines = list(csv_reader)
    list_sites_without_S2_data = []
    for item in list_lines:
        item = item[0].strip()
        if item != '': # check theri is no blank line
            if item != "sar_id_segment":
                list_sites_without_S2_data.append(item)
print(list_sites_without_S2_data)

In [ ]:
# Sélectionner les sites pour boucler dessus

def compo(dteDebut, dteFin, dteExe):

    
    # Define Database connection parameters
    # NOTE: password is in ~/.pgpass
    credentials = config(section='postgresql')

    db_credentials = {
        'host': credentials['host'],
        'user': credentials['user'],
        'db' : credentials['database']
    }

    # ALWAYS prepare env et the beginning
    print('> Preparing env (DB credentials, etc)')
    sarsar_admin.prepare_env(db_credentials)
    
    # Process all sar_id_segments
    conn = sarsar_admin._create_or_get_db_connection()
    cur = None
    cur2 = None

    try:
        import psycopg2.extras
        cur = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)
        cur2 = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)
        
        
        
#            strSQL = '''SELECT sar_id_segment, index_name, acq_date, index_mean, pixel_count 
#                        FROM sar_index_stats WHERE sar_id_segment = '{0}' AND index_name NOT IN ('BI2','VV','SBI','NDVI','BI2_part1','BI','BAI','VH') 
#                        AND substring(index_name,1,2) != 'VV' AND acq_date BETWEEN '{1}' AND '{2}' 
#                        ORDER BY sar_id_segment, index_name, acq_date;'''.format(site, dteDebut, dteFin)
        
        compteur = 0
        cur.execute('SELECT DISTINCT sar_id_segment FROM sar_index_stats;')
        result = cur.fetchall()    
        
        
        #result = ['52063-ISA-0020-01'] #DEBUG
        
        
        for row in result:
            sar_id_segment = dict(row)['sar_id_segment'] #Un-comment once debugged
            #sar_id_segment = '52063-ISA-0020-01'
#            sar_id_segment = '56078-ISA-0008-01'  # DEBUG compute_sar_compo_stats_by_sar_id error
#            sar_id_segment = '25072-ISA-0013-01'  # DEBUG
            compteur += 1
            print("> ",compteur,sar_id_segment)
            if sar_id_segment not in list_sites_without_S2_data:
                # Vérifie la disponibilité des données avant d'appeler la fonction sarsar_admin.compute_sar_compo_stats_by_sar_id;
                # car celle-ci appelle une procédure stockée de la db PostgreSQL ; qui lève une erreur lorsqu'il n'y a pas 
                # de données dans la table sar_index_stats pour le site, les indices et l'intervalle de dates demandés...
                ## select p.proname, p.prosrc from pg_proc as p where p.proname='compute_sar_compo_stats_by_sar_id';
                strSQL = '''SELECT count(*) FROM sar_index_stats
                            WHERE sar_id_segment = '{0}' AND acq_date BETWEEN '{1}' AND '{2}'
                            AND index_name IN ('BI', 'BI2', 'SBI');'''.format(sar_id_segment, dteDebut, dteFin)
                #print(strSQL)
                cur2.execute(strSQL)
                computables = cur2.fetchone()

                if computables[0] > 10: #If not enough data, results is useless (and std can't be calculated)
                    # Va chercher les composants de l'indice pour le site et l'intervalle de dates"
                    # (Retire les tirets des dates, pour convenir à la fonction compo_stats, )
                    
                    sarsar_admin.compute_sar_compo_stats_by_sar_id(sar_id_segment, dteDebut.replace('-', ''), dteFin.replace('-', ''))
                    compo_stats = sarsar_admin.fetch_sar_compo_stats_by_sar_id(sar_id_segment, dteDebut.replace('-', ''), dteFin.replace('-', '') )
                else:
                    compo_stats = []

#                print('The are %i records (acquisition dates with COMPO values) for this specific COMPO serie (sar_id_segment AND start_date_incl AND end_date_incl)' % len(compo_stats))

                # Calcule les compos "brutes"
                if len(compo_stats) > 0:
                    compo_ts = sarsar_admin.compute_compo_time_series_stats(compo_stats)

                    # Lisse la série de compos
                    smoothed_compo_ts = sarsar_admin.compute_smoothed_compo_time_series_stats(compo_stats, time_resampling='D', gaussian_sigma=61)

                    # smoothed_compo_ts is a tuple of 2 arrays: first is an array of interpolated dates, second is an array of corresponding smoothed stats
                    smoothed_profile_dico = {}

                    #for i in range(len(compo_stats)):
                    #print(len(compo_stats),len(smoothed_compo_ts[0]))
                    for i in range(len(smoothed_compo_ts[0])):
                        smoothed_profile_dico[smoothed_compo_ts[0][i].strftime("%Y-%m-%d")] = smoothed_compo_ts[1][i]

            #        pause = input("pressez une touche...")

                    # Enregistre la série smoothée dans la table "{sar_id_segment}_COMPO_{dteExe}_smoothed"
                    table_name = '{0}_COMPO_{1}_smoothed'.format(sar_id_segment, dteExe)

#                    print(table_name)  # DEBUG

                    strSQL = 'DROP TABLE IF EXISTS "{0}";'.format(table_name)  # Encadrer le nom de table pour tolérer les chiffres en début et les tirets au milieu
                    cur.execute(strSQL)
                    conn.commit()

                    strSQL = 'CREATE TABLE IF NOT EXISTS "{0}" (dte DATE, indice NUMERIC);'.format(table_name)  #, ', '.join(type_valeurs))
                    cur.execute(strSQL)

                    for item in smoothed_profile_dico:
                        strSQL = 'INSERT INTO "{0}" (dte, indice) VALUES (\'{1}\', {2});'.format(table_name, item, smoothed_profile_dico[item])  # , (', '.join(champs)), str(tuple(valeurs)))
                        cur.execute(strSQL)

                    # Poste toutes les modifications
                    conn.commit()
                else:
                    print("COMPO can't be computed : not enough data")
                
            else:
                print("segment ignoré")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        print('sar_id_segment =', sar_id_segment)
    finally:
        if cur is not None:
            cur.close()
            
    # ALWAYS release env at the end
    print('> Releasing env')
    sarsar_admin.release_env()
            

In [ ]:
dates = config(section='dates')
dteDebut = dates['deb']
dteFin = dates['fin']
dteExe = dates['exe']


#dteDebut = '20200101'
#dteFin = '20210601'

#dteDebut = '20171201'
#dteFin = '20220131'

compo(dteDebut, dteFin, dteExe)

In [ ]:
def smooth_time_serie(dteDebut, dteFin, dteExe):
    
    # List all indices
    indices = ['BAI','BI','BI2','NDVI','NDWI2','SBI']
        
    # Define Database connection parameters
    # NOTE: password is in ~/.pgpass
    credentials = config(section='postgresql')

    db_credentials = {
        'host': credentials['host'],
        'user': credentials['user'],
        'db' : credentials['database']
    }

    # ALWAYS prepare env et the beginning
    print('> Preparing env (DB credentials, etc)')
    sarsar_admin.prepare_env(db_credentials)
    
    # Process all sar_id_segments
    conn = sarsar_admin._create_or_get_db_connection()
    cur = None

    try:
        import psycopg2.extras
        cur = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)
        cur.execute('SELECT DISTINCT sar_id_segment FROM sar_index_stats;')    
        result = cur.fetchall()
        for row in result:
            sar_id_segment = dict(row)
            
            
            for indice in indices :
                print('> Processing smoothing for %s, %s' % (sar_id_segment, indice))
                raw_profile = sarsar_admin.fetch_sar_index_stats_by_sar_id(sar_id_segment['sar_id_segment'], indice, dteDebut, dteFin )
                smoothed_profile = sarsar_admin.compute_smoothed_time_series_stats(raw_profile)  # .remove_outliers_from_stats()

                # transpose les deux arrays dans un dictionnaire tel qu'attendu pour la boucle suivante...
                smoothed_dates = smoothed_profile[0]
                smoothed_values = smoothed_profile[1]
                smoothed_profile_dico = {}

                for i in range(len(smoothed_dates)):
                    smoothed_profile_dico[smoothed_dates[i].strftime("%Y-%m-%d")] = smoothed_values[i]

                # (re)crée LA table "{sar_id_segment}_{indice}_{dteExe}_smoothed" qui doit accueillir ces données
                table_name = '{0}_{1}_{2}_smoothed'.format(sar_id_segment['sar_id_segment'], indice, dteExe)
                strSQL = 'DROP TABLE IF EXISTS "{0}";'.format(table_name)  # Encadrer le nom de table pour tolérer les chiffres en début et les tirets au milieu
                cur.execute(strSQL)
                conn.commit()

                strSQL = 'CREATE TABLE IF NOT EXISTS "{0}" (dte DATE, indice NUMERIC);'.format(table_name)  #, ', '.join(type_valeurs))
                cur.execute(strSQL)

                for item in smoothed_profile_dico:
                    strSQL = 'INSERT INTO "{0}" (dte, indice) VALUES (\'{1}\', {2});'.format(table_name, item, smoothed_profile_dico[item])  # , (', '.join(champs)), str(tuple(valeurs)))
                    cur.execute(strSQL)

            # Poste toutes les modifications
            conn.commit()
            
        cur.close()
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
            
    # ALWAYS release env at the end
    print('> Releasing env')
    sarsar_admin.release_env()